# MLP with PyTorch

At this point, I know what MLPs are. We built one from scratch one of the previous folder. In this notebook, I want to implement an MLP again, but using PyTorch. This MLP will trained and evaluated on the MNIST dataset. 

# The MNIST dataset

In this section, I want to set the dataset in a way that can be used directly by our soon to be created MLP.